In [1]:
import os
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from dog_images import DogImages
from image_classifier import ImageClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [2]:
picsize = 128
# Step 1: Get List of Dogs
lst = [x[0] for x in os.walk('../Images')]
lst_dogs = [a.replace('../Images/', '') for a in lst[1:]]
print(lst_dogs[0])

n02097474-Tibetan_terrier


In [3]:
# Step 2: Make the Images...
dog_images = DogImages(lst_dogs, picsize)
# dog_images.generate_img_files()
train_imgs = dog_images.load_images('train')
test_imgs = dog_images.load_images('test')
Xtest = test_imgs[0]
Ytest = test_imgs[1]
Xtrain = train_imgs[0]
Ytrain = train_imgs[1]
print(lst_dogs[-1])

n02096177-cairn


In [4]:
# Step 3: Initial Shuffle of Train & Test Sets
Xhold = Xtrain.copy()
Yhold = Ytrain.copy()
new = np.array([i for i in range(Xhold.shape[0])])
np.random.shuffle(new)
for i, n in enumerate(new):
    Xtrain[i, :] = Xhold[n, :]
    Ytrain[i, :] = Yhold[n, :]

Xhold = Xtest
Yhold = Ytest
new = np.array([i for i in range(Xhold.shape[0])])
np.random.shuffle(new)
for i, n in enumerate(new):
    Xtest[i, :] = Xhold[n, :]
    Ytest[i, :] = Yhold[n, :]
print('done')

done


In [5]:
def shuffle_function(X, y):
    Xhold = X.copy()
    Yhold = y.copy()
    new = np.array([i for i in range(Xhold.shape[0])])
    np.random.shuffle(new)
    for i, n in enumerate(new):
        X[i, :] = Xhold[n, :]
        y[i, :] = Yhold[n, :]
    return X, y

In [6]:
def plot_learning(model):
    plt.plot(list(range(len(model.loss_function))),
             model.loss_function, color='y', label='Loss/Max Loss')
    plt.plot(list(range(len(model.train_accuracies))),
             (1/len(model.classes))*np.ones(len(model.train_accuracies)),
             linestyle='-', label='chance')
    plt.plot(list(range(len(model.train_accuracies))),
             model.train_accuracies, color='r', label='Training')
    plt.plot(list(range(len(model.val_accuracies))),
             model.val_accuracies, color='b', label='Validation')
    plt.ylabel('Probability')
    plt.xlabel('Epochs')
    plt.title('Accuracy & Loss')
    plt.ylim(ymax=1)
    plt.ylim(ymin=0)
    plt.legend()
    plt.show()

In [7]:
N = 27

In [8]:
# need 0:1 to get 0... it's a weird numpy thing
# so for lst_dogs[N], go Yt...[:, N:N + 1] 
Ytrain_1 = np.zeros((Ytrain.shape[0],2))
Ytest_1 = np.zeros((Ytest.shape[0],2))

Ytrain_1[Ytrain[:,N]==1] = [1, 0]
Ytrain_1[Ytrain[:,N]==0] = [0, 1]

Ytest_1[Ytest[:,N]==1] = [1, 0]
Ytest_1[Ytest[:,N]==0] = [0, 1]


In [9]:
Ytrain_a = Ytrain_1[Ytrain_1[:,0] == 1]
Xtrain_a = Xtrain[Ytrain_1[:,0] == 1]
Ytrain_b = Ytrain_1[Ytrain_1[:,0] == 0][:3*Ytrain_a.shape[0]]
Xtrain_b = Xtrain[Ytrain_1[:,0] == 0][:3*Ytrain_a.shape[0]]

In [10]:
Ytrain_run = np.concatenate((Ytrain_a, Ytrain_b))
Xtrain_run = np.concatenate((Xtrain_a, Xtrain_b))

In [11]:
Ytrain_run

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [12]:
lst_dogs_1 = [lst_dogs[N], 'None']
print(lst_dogs)

['n02097474-Tibetan_terrier', 'n02108089-boxer', 'n02100877-Irish_setter', 'n02115913-dhole', 'n02093256-Staffordshire_bullterrier', 'n02108422-bull_mastiff', 'n02093754-Border_terrier', 'n02110063-malamute', 'n02112018-Pomeranian', 'n02095314-wire-haired_fox_terrier', 'n02102973-Irish_water_spaniel', 'n02101388-Brittany_spaniel', 'n02109047-Great_Dane', 'n02098286-West_Highland_white_terrier', 'n02090721-Irish_wolfhound', 'n02096437-Dandie_Dinmont', 'n02112706-Brabancon_griffon', 'n02113186-Cardigan', 'n02105412-kelpie', 'n02093859-Kerry_blue_terrier', 'n02089973-English_foxhound', 'n02091635-otterhound', 'n02108551-Tibetan_mastiff', 'n02113799-standard_poodle', 'n02096051-Airedale', 'n02116738-African_hunting_dog', 'n02106382-Bouvier_des_Flandres', 'n02109961-Eskimo_dog', 'n02088238-basset', 'n02095570-Lakeland_terrier', 'n02100735-English_setter', 'n02110958-pug', 'n02102318-cocker_spaniel', 'n02111500-Great_Pyrenees', 'n02085620-Chihuahua', 'n02111889-Samoyed', 'n02095889-Sealyham_

In [ ]:

Xtrain_run, Ytrain_run = shuffle_function(Xtrain_run, Ytrain_run)
print(Ytrain_run)

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]


In [ ]:

lst_ch = [6,12,18,24]
lst_l = [0.01, 0.001, 0.0001]
lst_h = [300, 450, 600]
lst_reg = [0.25, 0.5, 0.75, 1.0]
lst_b = [50,75,100,200]
best_score = 0.0
best_i = 0
best_j = 0

for i in lst_b:
    for j in lst_ch:
        model = ImageClassifier(picsize, lst_dogs_1,
                                 out_channels = j,
                                 out_channels_2 = 2*j,
                                 hidden_units = 600,
                                 regularization_strength = 0.5,
                                 batch_size = i,
                                 learning_rate = 0.001,
                                 convolution_size = 5,
                                 pool_size = 2,
                                 training_epochs = 50,
                                 loss_threshold = 10.0,
                                 verbose=True,
                                 grid_search=True)
        model.fit(Xtrain_run, Ytrain_run)
        score = model.score(Xtest, Ytest_1)
        model.sess.close()
        print(' ', i, j, score)
        if score > best_score:
            best_score = score
            best_i = i
            best_j = j
        print(best_score,best_i,best_j)
print(best_score,best_i,best_j)

Percent Complete: 3.2857% - Train Accuracy: 83.673% - Validation Accuracy: 75.439% - Loss Function: 1375.5768

In [ ]:
model = ImageClassifier(picsize, lst_dogs,
                                 out_channels = 12,
                                 out_channels_2 = 24,
                                 hidden_units = 50,
                                 regularization_strength = 1.0,
                                 batch_size = 50,
                                 learning_rate = 0.0001,
                                 convolution_size = 5,
                                 pool_size = 2,
                                 training_epochs = 100,
                                 loss_threshold = 0.01,
                                 verbose=True,
                                 grid_search=True)

model.fit(Xtrain_run, Ytrain_run)
plot_learning(model)
score = model.score(Xtest, Ytest_1)
model.sess.close()
print(score)